# Words count with PySpark RDDs
If you've ever heard of "Hello, world!" for web development, "Word count" is the actual equivalent for distributed computing.

In this notebook, we will setup a pipeline that will also us to count the words of a document in a distributed manner. For convenience, we will do this on a single small document, but what we do should be able to generalize to bigger documents that would not fit into the memory of a single machine.

In [0]:
### BEGIN STRIP ###
import pyspark

spark = (pyspark.sql.SparkSession.builder \
         .master('local') \
         .appName('Introduction to PySpark') \
         .config("spark.some.config.option", "some-value") \
         .getOrCreate())

sc = spark.sparkContext

### END STRIP ###

In [0]:
# We need a S3 filepath
S3_RESOURCE = 's3'
SCHEME = 's3a'
# TODO: assign a BUCKET_NAME and PREFIX
BUCKET_NAME = ''
PREFIX = ''
FILENAME = 'purple_rain.txt'

In [0]:
# This is just a utility function
def get_s3_path(key, bucket_name=BUCKET_NAME, scheme=SCHEME):
    return f"{scheme}://{bucket_name}/{key}"

In [0]:
filepath = get_s3_path(FILENAME)
### BEGIN STRIP ###
# This is required for local work

### END STRIP ###

In [0]:
# TODO: it already maps to lines... ???
# TODO: load the filepath to a Spark RDD using `.textFile(...)` from a SparkContext
### BEGIN STRIP ###
from pathlib import Path
path = Path("FileStore", "shared_uploads", "thibaudchevrier@gmail.com", "purple_rain.txt")
purple_rain_rdd = sc.textFile(str(path))

### END STRIP ###

In [0]:
# TODO: print out `text_file`
### BEGIN STRIP ###

print(purple_rain_rdd)

### END STRIP ###

FileStore/shared_uploads/thibaudchevrier@gmail.com/purple_rain.txt MapPartitionsRDD[167] at textFile at <unknown>:0

That doesn't tell us much, how would you do to see the first 3 elements of this RDD?

In [0]:
# TODO: take the first 3 elements of the RDD `text_file`
### BEGIN STRIP ###

purple_rain_rdd.take(3)

### END STRIP ###

Out[87]: ['I never meant to cause you any sorrow',
 'I never meant to cause you any pain',
 'I only wanted one time to see you laughing']

This is a list of sentences, what we want is a list of tokens.

In [0]:
### BEGIN STRIP ###

tokenized_text = purple_rain_rdd.map(lambda line: line.split(" "))
tokenized_text.take(3)

### END STRIP ###

Out[88]: [['I', 'never', 'meant', 'to', 'cause', 'you', 'any', 'sorrow'],
 ['I', 'never', 'meant', 'to', 'cause', 'you', 'any', 'pain'],
 ['I', 'only', 'wanted', 'one', 'time', 'to', 'see', 'you', 'laughing']]

That's not exactly what we wanted... We wanted a list of tokens, we got a.. **list of list of tokens**.  
That's because, in this case, we need a special version of `.map()` called `flatMap`: it will flatten the list of list of tokens into a list of tokens.

Let's try it out: we take the same expression as the previous one, but replace `.map()` with `.flatMap()` and call the resulting variable `tokens`.

---
💡 It usually takes time to understand the notion of `.flatMap` and flattening in general, like `.map()`, these are concepts from the functionnal programming world. Unless you come from such background, it probably **won't be easy to grasp these concepts the first time to encouter them**.

**Let's keep our eyes on the ball: our goal today is not to understand the specifics of these, but to develop a broader understanding of how Spark works.**

---

In [0]:
# TODO: copy/paste the previous cell, and:
# - replace `.map(...) with `.flatMap(...)`
# - rename the variable `tokenized_text` to `tokens`
### BEGIN STRIP ###

tokens = purple_rain_rdd.flatMap(lambda line: line.split(" "))

### END STRIP ###

In [0]:
# TODO: Use this cell to play with `tokens`, take different amounts of it, or collect it.
### BEGIN STRIP ###

tokens.take(10)

### END STRIP ###

Out[90]: ['I', 'never', 'meant', 'to', 'cause', 'you', 'any', 'sorrow', 'I', 'never']

Now that we have our list of words (well, **not exactly a list of words, it is still a RDD**), we can start counting things.

In order to do that, we need to map each word to an initial count, so instead of having:
```
['I',
 'never',
 'meant',
 ...,
 'I',
 'never',
 ...]
```
We would like our list to look like this:
```
[('I', 1),
 ('never', 1),
 ('meant', 1),
 ...,
 ('I', 1),
 ('never', 1),
 ...]
```

In [0]:
# TODO: Write a function `token_to_tuple` that takes:
#       - a token as input (a string)
#       - and returns (token, 1) (a tuple) 
### BEGIN STRIP ###

def token_to_tuple(token):
  return (token, 1)
  

### END STRIP ###

In [0]:
# TODO: map `tokens` to your new function `token_to_tuple`: `partial_count`
### BEGIN STRIP ###
partial_count = tokens.map(lambda token: token_to_tuple(token))
### END STRIP ###

In [0]:
# TODO: take the first 10 elements of `partial_count`
### BEGIN STRIP ###
partial_count.take(10)
### END STRIP ###

Out[93]: [('I', 1),
 ('never', 1),
 ('meant', 1),
 ('to', 1),
 ('cause', 1),
 ('you', 1),
 ('any', 1),
 ('sorrow', 1),
 ('I', 1),
 ('never', 1)]

Good job!

Beware, now comes the hard-part... We need to reduce this..

Don't forget, when we start using DataFrame, because these are higher level abstractions, it will take care of most of these steps for us.

What we want, is take the tuple with similar keys, like `('never', 1)` and `('never', 1)` and count them, so in the end we have `('never', 2)` (or more than 2 if there are more occurence of 'never').

These kind of tuples are called **key-value pairs**, and while most Spark operations work on RDDs containing any type of objects, a few special operations are only available on RDDs of key-value pairs. You can read more about it [in the documentation](https://spark.apache.org/docs/latest/rdd-programming-guide.html#working-with-key-value-pairs).

Among these operations are `.groupByKey(...)` and `.reduceByKey(...)`: the latter has better performances, but the former is easier to understand so we will start with this one.

### groupByKey

In [0]:
# TODO: call `.groupByKey(...)` on partial_count: grouped_by_key
#       and take the first 3 elements
### BEGIN STRIP ###
grouped_by_key = partial_count.groupByKey()
### END STRIP ###

In [0]:
# TODO: take the first 3 elements of `grouped_by_key`
### BEGIN STRIP ###
grouped_by_key.take(3)
### END STRIP ###

Out[95]: [('never', <pyspark.resultiterable.ResultIterable at 0x7f691cd13610>),
 ('cause', <pyspark.resultiterable.ResultIterable at 0x7f691cd13790>),
 ('pain', <pyspark.resultiterable.ResultIterable at 0x7f691cd15e50>)]

What's this: `<pyspark.resultiterable.ResultIterable at 0x10bc0c2d0>` ?

You don't have to worry about the details, but one thing has to attract your attention: `Iterable`, this seems to suggest those objects are iterable, an iterable in Python is something that you can iterate on: basicall something that you can call `for` on, like a list, or a string, etc..

```
for letter in 'Spark':
    print(e)
> S
> p
> a
> r
> k
```

Each element of `grouped_by_key` is a tuple, and inside a tuple there is an iterable we can iterate over.

We will first try with the first element.

In [0]:
# TODO: take the first element of `grouped_by_key`: first_item
#       and print out its type
# WARNING: the type should be a tuple, not a list
### BEGIN STRIP ###
type(grouped_by_key.take(1)[0])
### END STRIP ###

Out[96]: tuple

We'd like a way to print these items, for example, such that 'never' would look like this:
```
'never': [1, 1, 1, 1]
```

We will write a function that does this, take an item as a tuple of (`str`, `ResultIterable`), and print out:
```
ITEM_NAME: OCCURENCES_AS_A_LIST
```

In [0]:
# TODO: define a way to print our item
### BEGIN STRIP ###
def print_item(token):
  print(token[0], [value for value in token[1]])

### END STRIP ###

In [0]:
# TODO: take the first 10 items from grouped_by_key and then iterate over them
#       then, inside the loop, use the function `print_item(...)` on each item
### BEGIN STRIP ###
for token in grouped_by_key.take(10):
  print_item(token)
### END STRIP ###

never [1, 1, 1, 1]
cause [1, 1]
pain [1]
only [1, 1, 1, 1, 1, 1, 1]
in [1, 1]
rain [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Purple [1, 1, 1, 1, 1, 1, 1, 1, 1]
rain, [1, 1, 1, 1, 1, 1, 1, 1, 1]
bathing [1]

Next step might be challenging.

When you take the first 10 elements of `grouped_by_key`, it returns a list of `Tuple[str, ResultIterable]`.  
What we want instead is a list of `Tuple[str, int]` where the second element is the total number of occurence for the fist element.

NOTE: you might wanna try to first return a list of `Tuple[str, list]`.

You should be able to do all this using only list comprehensions.

In [0]:
# TODO: follow previous instructions
### BEGIN STRIP ###
grouped_by_key.map(lambda token: (token[0], sum([value for value in token[1]]))).collect()
### END STRIP ###

Out[99]: [('never', 4),
 ('cause', 2),
 ('pain', 1),
 ('only', 7),
 ('in', 2),
 ('rain', 14),
 ('', 10),
 ('Purple', 9),
 ('rain,', 9),
 ('bathing', 1),
 ('kind', 1),
 ('of', 1),
 ('steal', 1),
 ("It's", 2),
 ('end', 1),
 ('underneath', 1),
 ('know', 2),
 ('are', 1),
 ('we', 1),
 ('out', 1),
 ('something', 1),
 ('new', 1),
 ('That', 1),
 ('means', 1),
 ('say', 1),
 ('leader', 1),
 ('But', 1),
 ('make', 1),
 ('mind', 1),
 ('think', 1),
 ('close', 1),
 ('And', 1),
 ('let', 1),
 ('guide', 1),
 ('singing', 1),
 ("C'mon,", 1),
 ('I', 14),
 ('meant', 2),
 ('to', 13),
 ('you', 14),
 ('any', 2),
 ('sorrow', 1),
 ('wanted', 3),
 ('one', 1),
 ('time', 2),
 ('see', 6),
 ('laughing', 2),
 ('want', 6),
 ('the', 5),
 ('purple', 14),
 ('be', 2),
 ('your', 3),
 ('weekend', 1),
 ('lover', 1),
 ('some', 1),
 ('friend', 1),
 ('Baby,', 1),
 ('could', 1),
 ('from', 1),
 ('another', 1),
 ('such', 1),
 ('a', 2),
 ('shame', 1),
 ('our', 1),
 ('friendship', 1),
 ('had', 1),
 ('Honey', 1),
 ('know,', 2),
 ('times', 1),
 ('changing', 1),
 ('all', 1),
 ('reach', 1),
 ('for', 1),
 ('too', 1),
 ('You', 1),
 ("can't", 1),
 ('seem', 1),
 ('up', 2),
 ('better', 1),
 ('it', 1),
 ('me', 1),
 ('If', 1),
 ('what', 1),
 ("I'm", 1),
 ('about', 1),
 ('here', 1),
 ('raise', 1),
 ('hand', 1),
 ('you,', 1),
 ('In', 1)]

As you've seen this can be done using standard list comprehension.  
If you're curious, even though Python is not a purely functional language, you can write this in a functional fashion and achieve the same result.

_Please note this would look obviouly more elegant in a purely functional language.  
And I put it in there only to introduce you to another programming paradigm if you've mostly encountered imperative programming before. This little introduction is helpful because Spark is based on Scala, which, although not being a purely functional language, provides support for many functional programming features._

In [0]:
# Don't try at home ;)

from functools import reduce

list(map(lambda t: (t[0], reduce(lambda a, b: a + b, t[1])),
         grouped_by_key.take(10)))

Out[100]: [('never', 4),
 ('cause', 2),
 ('pain', 1),
 ('only', 7),
 ('in', 2),
 ('rain', 14),
 ('', 10),
 ('Purple', 9),
 ('rain,', 9),
 ('bathing', 1)]

That would work, but that's using regular Python, hence we're not profiting from Spark's distributed computing capabilities, which means:
- the computation would be much slower on big datasets
- if the datasets is too big to be stored on the memory of our machine our program would crash

That's exactly what `.reduceByKey(...)` will help us to solve. It's usage is a bit similar to `.groupByKey(...)` but it takes a function as a parameter, this function should tell Spark how to aggregate 2 items, in our case, that the value of each tuple, for example :
Let's say we have a group:
```
('dog', 1), ('dog', 1)
```
we want a formula applied on values that will give us the end result, e.g. "how many dogs".
In our case, that's a simple sum:
```
def reduce_function(value_1, value_2):
    return value_1 + value_2
```

In [0]:
# TODO: write our reduce function: reduce_function
#       which takes 2 values and return their sum
# NOTE: name this parameters `a` and `b`
### BEGIN STRIP ###
def reduce_function(a, b):
    return a + b
### END STRIP ###

We're now ready to reduce. You will pass your function as parameter to `.reduceByKey(...)`.

In [0]:
# TODO: call `.reduceByKey(...) on `partial_count`: `reduced`
### BEGIN STRIP ###
reduced = partial_count.reduceByKey(reduce_function)
### END STRIP ###

In [0]:
# TODO: take the 10 first values
### BEGIN STRIP ###
reduced.take(10)
### END STRIP ###

Out[103]: [('never', 4),
 ('cause', 2),
 ('pain', 1),
 ('only', 7),
 ('in', 2),
 ('rain', 14),
 ('', 10),
 ('Purple', 9),
 ('rain,', 9),
 ('bathing', 1)]

**[TODO]: reword this part**

Wow! Good job. We're almost there... 😅
We've got a list of tuples, where the key is the token, and the value is its count within the text, but.. 
**they're not ordered...** which is inconvenient if we want to have the 10 most popular tokens within the text.

We will use `.sortBy(...)`, but before we do, let's have a refresher on sorting with Python.

For example, how would you sort this grocery list by the number of items?

In [0]:
fruits = [('banana', 3), ('orange', 5), ('pineapple', 2)]
fruits

Out[104]: [('banana', 3), ('orange', 5), ('pineapple', 2)]

`sorted(fruits)` won't work because by default sorting on tuple take the first element, in our case, it would sort alphabetically on the name of the fruits.

In [0]:
sorted(fruits)

Out[105]: [('banana', 3), ('orange', 5), ('pineapple', 2)]

We can force the `key` parameter to sort on the second item of each tuple.

In [0]:
sorted(fruits, key=lambda x: x[1])

Out[106]: [('pineapple', 2), ('banana', 3), ('orange', 5)]

Now, we will do the same on our rdd. Just like `key` in Python's `sorted`, PySpark's `.sortBy(...)` can take a function as a parameter.

In [0]:
# TODO: use `.sortBy(...)` on `reduced`: sorted_counts
### BEGIN STRIP ###
sorted_counts = reduced.sortBy(lambda t: t[1])
### END STRIP ###

In [0]:
# TODO: take the 10 first values of `sorted_counts`
### BEGIN STRIP ###
sorted_counts.take(10)
### END STRIP ###

Out[108]: [('pain', 1),
 ('bathing', 1),
 ('kind', 1),
 ('of', 1),
 ('steal', 1),
 ('end', 1),
 ('underneath', 1),
 ('are', 1),
 ('we', 1),
 ('out', 1)]

What do you think?

It seems sorted, but in **ascending order**..  
If we wanted to do this in Python, we could just set the `reverse` argument to `True` when calling `sorted(...)`.

In [0]:
sorted(fruits, key=lambda x: x[1], reverse=True)

Out[109]: [('orange', 5), ('banana', 3), ('pineapple', 2)]

We can't do this with Spark's RDDs. What we can do instead is **take the opposite value and order by it**.

In [0]:
# TODO: use `.sortBy(...)` on `reduced`, but with a descending sort: desc_sorted_counts
### BEGIN STRIP ###
desc_sorted_counts = reduced.sortBy(lambda t: t[1], False)
### END STRIP ###

In [0]:
# TODO: take the 10 first values of `desc_sorted_counts`
### BEGIN STRIP ###
desc_sorted_counts.take(10)
### END STRIP ###

Out[111]: [('rain', 14),
 ('I', 14),
 ('you', 14),
 ('purple', 14),
 ('to', 13),
 ('', 10),
 ('Purple', 9),
 ('rain,', 9),
 ('only', 7),
 ('see', 6)]

Finally, what's the most common word in our document?

### **Bonus**: putting everything together

We will create a function `count_words` that will do everything we did previously, but this time in one swell swoop, we won't use intermediary variables.

The function will:
- take a filepath as argument
- load the content of this filepath into a Spark RDD
- `flatMap(...)` each line of this RDD into tokens by splitting on the ' ' string
- `.map(...)` each token to `(token, 1)` so this can be then reduced
- by calling `.reduceByKey(...)` with a function that sums the values
- and then sort the results with `.sortBy(...)` using the proper function to sort in descending order
- and return an RDD

---
⚠️ Make sure your function returns a RDD

---

In [0]:
def count_words(filepath):
    # TODO: implement the content of the function
    # 
    # NOTE: you can remove `pass`
    # it's just here to avoid the cell crashing while the
    # content of the function is empty
    ### BEGIN STRIP ###
    text_rdd = sc.textFile(str(filepath))
    return text_rdd.flatMap(lambda line: [(t, 1) for t in line.split(" ")]).reduceByKey(lambda a, b: a+b).sortBy(lambda t: t[1], False)
    ### END STRIP ###

In [0]:
# TODO: use `count_words` on `filepath` and check its type
### BEGIN STRIP ###
res = count_words(path)
print(res.collect())
### END STRIP ###

[('rain', 14), ('I', 14), ('you', 14), ('purple', 14), ('to', 13), ('', 10), ('Purple', 9), ('rain,', 9), ('only', 7), ('see', 6), ('want', 6), ('the', 5), ('never', 4), ('wanted', 3), ('your', 3), ('cause', 2), ('in', 2), ("It's", 2), ('know', 2), ('meant', 2), ('any', 2), ('time', 2), ('laughing', 2), ('be', 2), ('a', 2), ('know,', 2), ('up', 2), ('pain', 1), ('bathing', 1), ('kind', 1), ('of', 1), ('steal', 1), ('end', 1), ('underneath', 1), ('are', 1), ('we', 1), ('out', 1), ('something', 1), ('new', 1), ('That', 1), ('means', 1), ('say', 1), ('leader', 1), ('But', 1), ('make', 1), ('mind', 1), ('think', 1), ('close', 1), ('And', 1), ('let', 1), ('guide', 1), ('singing', 1), ("C'mon,", 1), ('sorrow', 1), ('one', 1), ('weekend', 1), ('lover', 1), ('some', 1), ('friend', 1), ('Baby,', 1), ('could', 1), ('from', 1), ('another', 1), ('such', 1), ('shame', 1), ('our', 1), ('friendship', 1), ('had', 1), ('Honey', 1), ('times', 1), ('changing', 1), ('all', 1), ('reach', 1), ('for', 1), ('too', 1), ('You', 1), ("can't", 1), ('seem', 1), ('better', 1), ('it', 1), ('me', 1), ('If', 1), ('what', 1), ("I'm", 1), ('about', 1), ('here', 1), ('raise', 1), ('hand', 1), ('you,', 1), ('In', 1)]

It should be a `pyspark.rdd.PipelineRDD`

In [0]:
# TODO: finally, take the 10 first elements of your RDD
### BEGIN STRIP ###
res.take(10)
### END STRIP ###

Out[114]: [('rain', 14),
 ('I', 14),
 ('you', 14),
 ('purple', 14),
 ('to', 13),
 ('', 10),
 ('Purple', 9),
 ('rain,', 9),
 ('only', 7),
 ('see', 6)]

That's it, you've done it!

You've created a Spark job, the next step would be to neatly package this into a Python executable and submit it to a Spark Cluster for batch or stream execution, but this is beyond the content of this course.

## Going further

We used a toy dataset, we suggest you try with a bigger one.